# Visualize MTA GTFS feeds
inspired by

https://github.com/Bondify/gtfs_functions#installation
 


# download data

In [1]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install gtfs_functions

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import re

In [4]:
url = 'http://web.mta.info/developers/files/google_transit_supplemented.zip'
urllib.request.urlretrieve (url, "GTFS_nyc_Subway.zip")

('GTFS_nyc_Subway.zip', <http.client.HTTPMessage at 0x7f981174c210>)

In [5]:
'''zip_ref = zipfile.ZipFile("GTFS_nyc_Subway.zip", 'r')
zip_ref.extractall("GTFS_nyc_Subway")
zip_ref.close()'''

'zip_ref = zipfile.ZipFile("GTFS_nyc_Subway.zip", \'r\')\nzip_ref.extractall("GTFS_nyc_Subway")\nzip_ref.close()'

Installation

In [6]:
import gtfs_functions as gtfs
# import geopandas

In [7]:
routes, stops, stop_times, trips, shapes = gtfs.import_gtfs(r"GTFS_nyc_Subway.zip")

In [8]:
cutoffs = [0,6,9,15.5,19,22,24]
stop_freq = gtfs.stops_freq(stop_times, stops, cutoffs = cutoffs)
stop_freq.head(2)

,stop_id,dir_id,window,ntrips,frequency,max_trips,max_freq,stop_name,geometry
2112,901N,Inbound,0:00-6:00,2,180,18,3,Grand Central-42 St,POINT (-73.97919 40.75277)
2118,901S,Outbound,0:00-6:00,2,180,19,3,Grand Central-42 St,POINT (-73.97919 40.75277)


In [9]:
cutoffs = [0,6,9,15.5,19,22,24]
line_freq = gtfs.lines_freq(stop_times, trips, shapes, routes, cutoffs = cutoffs)
line_freq.head()

,route_id,route_name,dir_id,window,frequency,ntrips,max_freq,max_trips,geometry
752,C,C 8 Avenue Local,Inbound,0:00-6:00,360,1,8,7,"LINESTRING (-73.87211 40.67538, -73.88075 40.6..."
722,B,B 6 Avenue Express,Inbound,0:00-6:00,360,1,7,8,"LINESTRING (-73.96138 40.57762, -73.96035 40.5..."
723,B,B 6 Avenue Express,Inbound,0:00-6:00,360,1,7,8,"LINESTRING (-73.96138 40.57762, -73.96035 40.5..."
724,B,B 6 Avenue Express,Inbound,0:00-6:00,360,1,7,8,"LINESTRING (-73.96138 40.57762, -73.96035 40.5..."
725,B,B 6 Avenue Express,Inbound,0:00-6:00,360,1,7,8,"LINESTRING (-73.96138 40.57762, -73.96035 40.5..."


In [10]:
segments_gdf = gtfs.cut_gtfs(stop_times, stops, shapes)
segments_gdf.head(2)

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



,route_id,direction_id,stop_sequence,start_stop_name,end_stop_name,start_stop_id,end_stop_id,segment_id,shape_id,geometry,distance_m
0,1,0,1,South Ferry,Rector St,142N,139N,142N-139N,1..N03R,"LINESTRING (-74.01366 40.70207, -74.01479 40.7...",659.980740
1,1,0,2,Rector St,WTC Cortlandt,139N,138N,139N-138N,1..N03R,"LINESTRING (-74.01378 40.70751, -74.01347 40.7...",499.195737


In [11]:
# Cutoffs to make get hourly values
cutoffs = list(range(24))
speeds = gtfs.speeds_from_gtfs(routes, stop_times, segments_gdf, cutoffs = cutoffs)
speeds.head(1)

,route_id,route_name,dir_id,segment_id,window,speed_kmh,s_st_id,s_st_name,e_st_id,e_st_name,distance_m,stop_seq,runtime_h,max_kmh,geometry,speed_mph,max_mph
0,1,1 Broadway - 7 Avenue Local,Inbound,142N-139N,0:00-1:00,26.0,142N,South Ferry,139N,Rector St,659.98074,1,0.025,26.0,"LINESTRING (-74.01366 40.70207, -74.01479 40.7...",16.155646,16.155646


In [12]:
speeds.loc[(speeds.segment_id=='142N-139N')&(speeds.window=='0:00-1:00')]

,route_id,route_name,dir_id,segment_id,window,speed_kmh,s_st_id,s_st_name,e_st_id,e_st_name,distance_m,stop_seq,runtime_h,max_kmh,geometry,speed_mph,max_mph
0,1,1 Broadway - 7 Avenue Local,Inbound,142N-139N,0:00-1:00,26.0,142N,South Ferry,139N,Rector St,659.98074,1,0.025,26.0,"LINESTRING (-74.01366 40.70207, -74.01479 40.7...",16.155646,16.155646
102071,ALL_LINES,All lines,NA,142N-139N,0:00-1:00,26.0,142N,South Ferry,139N,Rector St,659.98074,1,0.025,NaN,"LINESTRING (-74.01366 40.70207, -74.01479 40.7...",16.155646,NaN


In [64]:
cutoffs = [0,6,9,15.5,19,22,24]
seg_freq = gtfs.segments_freq(segments_gdf, stop_times, routes, cutoffs = cutoffs)
seg_freq.head(2)

,route_id,route_name,dir_id,segment_id,window,frequency,ntrips,s_st_id,s_st_name,e_st_name,max_freq,max_trips,geometry
8696,Q,Q Broadway Express,Inbound,N05N-N04N,9:00-15:30,390,1,N05N,18 Av,New Utrecht Av,30,2,"LINESTRING (-73.99041 40.62067, -73.99515 40.6..."
8705,Q,Q Broadway Express,Inbound,N08N-N07N,9:00-15:30,390,1,N08N,Kings Hwy,Bay Pkwy,20,3,"LINESTRING (-73.98035 40.60392, -73.98185 40.6..."


In [65]:
seg_freq.loc[(seg_freq.segment_id=='N08N-N07N')&(seg_freq.window=='0:00-6:00')]

,route_id,route_name,dir_id,segment_id,window,frequency,ntrips,s_st_id,s_st_name,e_st_name,max_freq,max_trips,geometry
8019,N,N Broadway Local,Inbound,N08N-N07N,0:00-6:00,18,20,N08N,Kings Hwy,Bay Pkwy,7,8,"LINESTRING (-73.98035 40.60392, -73.98185 40.6..."
10117,ALL_LINES,All lines,NA,N08N-N07N,0:00-6:00,18,20,N08N,Kings Hwy,Bay Pkwy,6,40,"LINESTRING (-73.98035 40.60392, -73.98185 40.6..."


In [15]:
pip install branca

Note: you may need to restart the kernel to use updated packages.


In [16]:
# Stops
condition_dir = stop_freq.dir_id == 'Inbound'
condition_window = stop_freq.window == '0:00-6:00'

gdf = stop_freq.loc[(condition_dir & condition_window),:].reset_index()

gtfs.map_gdf(gdf = gdf, 
              variable = 'ntrips', 
              colors = ["#d13870", "#e895b3" ,'#55d992', '#3ab071', '#0e8955','#066a40'], 
              tooltip_var = ['frequency'] , 
              tooltip_labels = ['Frequency: '], 
              breaks = [10, 20, 30, 40, 120, 2000])

In [17]:
# Line frequencies
condition_dir = line_freq.dir_id == 'Inbound'
condition_window = line_freq.window == '6:00-9:00'

gdf = line_freq.loc[(condition_dir & condition_window),:].reset_index()

seg_map = gtfs.map_gdf(gdf = gdf, 
              variable = 'ntrips', 
              colors = ["#d13870", "#e895b3" ,'#55d992', '#3ab071', '#0e8955','#066a40'], 
              tooltip_var = ['route_name'] , 
              tooltip_labels = ['Route: '], 
              breaks = [1, 15, 20, 30])

seg_map

In [37]:
type(seg_map)

folium.folium.Map

In [22]:
b = speeds.drop(speeds[speeds['route_id']=='ALL_LINES'].index)
b = b[['geometry','speed_mph']]

In [40]:
import shapely

In [41]:
b['wkt'] = b.geometry.apply(lambda x: shapely.wkt.dumps(x))

In [42]:
b = b[['wkt','speed_mph']]

In [20]:
pip install keplergl

Note: you may need to restart the kernel to use updated packages.


In [45]:
# Speeds
import keplergl as kp
w = kp.KeplerGl(data = {"speed data":b}, height=800)
w

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'speed data':                                                       wkt  speed_mph
0       LINE…

In [59]:
seg_freq['geometry'] = seg_freq['geometry'].apply(lambda x: str(x))

In [67]:
seg_freq = seg_freq[['geometry','max_trips']]
seg_freq['wkt'] = seg_freq.geometry.apply(lambda x: shapely.wkt.dumps(x))

In [68]:
# Segment frequencies
import keplergl as kp
k = kp.KeplerGl(data=dict(data=seg_freq, name='Segment frequency'), height=800)
k

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'data':                                                 geometry  max_trips  \
8696   LINESTRIN…

In [ ]:
speeds

,route_id,route_name,dir_id,segment_id,window,speed_kmh,s_st_id,s_st_name,e_st_id,e_st_name,distance_m,stop_seq,runtime_h,max_kmh,geometry,speed_mph,max_mph
0,1,1 Broadway - 7 Avenue Local,Inbound,142N-139N,0:00-1:00,26.0,142N,South Ferry,139N,Rector St,659.980740,1,0.025000,26.0,"LINESTRING (-74.01366 40.70207, -74.01479 40.7...",16.155646,16.155646
3,1,1 Broadway - 7 Avenue Local,Inbound,139N-138N,0:00-1:00,20.0,139N,Rector St,138N,WTC Cortlandt,499.195737,2,0.025000,20.0,"LINESTRING (-74.01378 40.70751, -74.01347 40.7...",12.427420,12.427420
6,1,1 Broadway - 7 Avenue Local,Inbound,138N-137N,0:00-1:00,29.0,138N,WTC Cortlandt,137N,Chambers St,478.450553,3,0.016667,29.0,"LINESTRING (-74.01219 40.71184, -74.01211 40.7...",18.019759,18.019759
9,1,1 Broadway - 7 Avenue Local,Inbound,137N-136N,0:00-1:00,19.0,137N,Chambers St,136N,Franklin St,477.632017,4,0.025000,35.0,"LINESTRING (-74.00927 40.71548, -74.00725 40.7...",11.806049,21.747985
12,1,1 Broadway - 7 Avenue Local,Inbound,136N-135N,0:00-1:00,24.0,136N,Franklin St,135N,Canal St,395.900275,5,0.016667,24.0,"LINESTRING (-74.00689 40.71932, -74.00628 40.7...",14.912904,14.912904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125495,ALL_LINES,All lines,NA,A61S-A63S,9:00-10:00,16.0,A61S,Rockaway Blvd,A63S,104 St,541.052975,49,0.033333,NaN,"LINESTRING (-73.84385 40.68043, -73.84169 40.6...",9.941936,NaN
125496,ALL_LINES,All lines,NA,255S-256S,9:00-10:00,28.0,255S,Pennsylvania Av,256S,Van Siclen Av,473.642761,50,0.016667,NaN,"LINESTRING (-73.89490 40.66463, -73.88939 40.6...",17.398388,NaN
125497,ALL_LINES,All lines,NA,A63S-A64S,9:00-10:00,16.7,A63S,104 St,A64S,111 St,550.845206,50,0.036111,NaN,"LINESTRING (-73.83768 40.68171, -73.83759 40.6...",10.376896,NaN
125498,ALL_LINES,All lines,NA,256S-257S,9:00-10:00,18.0,256S,Van Siclen Av,257S,New Lots Av,457.776212,51,0.025000,NaN,"LINESTRING (-73.88939 40.66545, -73.88408 40.6...",11.184678,NaN


In [ ]:
# Heatmap
import plotly.express as px
import plotly.graph_objects as go
dir_0 = speeds.loc[(speeds.dir_id=='Inbound')&(speeds.route_name=='1 Broadway - 7 Avenue Local')].sort_values(by='stop_seq') 
dir_0['hour'] = dir_0.window.apply(lambda x: int(x.split(':')[0]))
dir_0.sort_values(by='hour', ascending=True, inplace=True)

fig = go.Figure(data=go.Heatmap(
                   z=dir_0.speed_kmh,
                   y=dir_0.s_st_name,
                   x=dir_0.window,
                   hoverongaps = False,
                   colorscale=px.colors.colorbrewer.RdYlBu, 
                   reversescale=False
))

fig.update_yaxes(title_text='Stop', autorange='reversed')
fig.update_xaxes(title_text='Hour of day', side='top')
fig.update_layout(showlegend=False, height=1000, width=1000,
                 title='Speed heatmap per direction and hour of the day')

fig.show()